In [1]:
%matplotlib inline
import math
import pandas as pd
from tsmom_model import *
import numpy as np
from datetime import datetime
import math
import statsmodels.api as sm

/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
import pylab
sns.set(font_scale=2)
pylab.rcParams['figure.figsize'] = (24, 18)

# Hand cleansed
Hand cleaned to removed duplicates

In [3]:
cleansed=load_amihud_markets_price()

tsmom_model.py:158: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data=pd.read_csv('./Price.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')
tsmom_model.py:160: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  qd=pd.read_csv('./liquid_contracts.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')


# Amihud methodology

Amihud (2002) developed the illiquidity measure expressed by:

$$ Amihud =\frac{1}{N} \sum \frac{|r_t|}{\text{dvol}_t} $$

where $r_t$ is the return on day $t$ and $Volume_t$ is the dollar volume on day $t$. 

The benefit of taking the daily return is to weight the move by change of price and the volume.


In [4]:
amihud=calculate_amihud_liquidity(cleansed)[:'2016'] 

tsmom_model.py:42: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  data=pd.read_csv('./Volume.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='mean')
tsmom_model.py:44: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  v=pd.read_csv('./volume_data.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='sum')
tsmom_model.py:46: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=100,window=250,center=False).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:46: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:49: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future ve

In [5]:
FHT=calculate_FHT(cleansed)

tsmom_model.py:283: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  nonzero=x[x==0].resample(rule='m',how='count')
tsmom_model.py:284: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  Z=(nonzero/x.resample(rule='m',how='count')).dropna()
tsmom_model.py:285: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=12,center=False).std()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(rule='m',how='last')
tsmom_model.py:285: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(rule='m',how='last')


In [6]:
results=pd.DataFrame()

In [7]:
results['Mean Return']=(cleansed.resample(rule='m',how='last').pct_change().mean()*100).round(2)
results['Annualized Volatility']=(cleansed.resample(
    rule='m',how='last').pct_change().std()*math.sqrt(12)*100).round(2)
results['Skew']=cleansed.resample(rule='m',how='last').pct_change().skew().round(2)
results['Kurtosis']=cleansed.resample(rule='m',how='last').pct_change().kurtosis().round(2)
results['Amihud (x1e12)']=(amihud.median()*1e12).apply(lambda x: '%.3f' % x)
results['FHT (x1e4)']=(FHT.median()*1e4).round(2)

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  """Entry point for launching an IPython kernel.
/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.
/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  """


In [8]:
results.sort_index()

,Mean Return,Annualized Volatility,Skew,Kurtosis,Amihud (x1e12),FHT (x1e4)
AEX-AEX INDEX,0.42,21.62,-2.56,19.18,8.634,131.51
AUD,0.11,12.71,-0.31,1.21,10.889,111.10
BMF-ARABICA COFFEE,0.51,32.08,0.58,1.07,1947.088,233.60
BMF-BOVESPA INDEX,2.78,62.13,1.20,15.24,2.476,246.70
BMF-SOYBEAN,1.24,24.44,-0.43,0.43,8684.163,186.66
CAC 40,0.60,17.77,-0.10,-0.29,21.000,203.61
CAD,0.07,8.86,-0.31,2.28,9.565,82.18
CBT-10 YRS US T-NOTE COMP.,0.11,6.71,1.35,22.12,0.034,47.70
CBT-2 YEAR US T-NOTE COMP.,0.03,1.82,-0.99,8.30,0.064,11.27
CBT-30 YR US T-BOND COMP,0.14,11.72,-0.01,3.19,0.166,75.39


In [9]:
print results.sort_index().to_latex()

\begin{tabular}{lrrrrlr}
\toprule
{} &  Mean Return &  Annualized Volatility &  Skew &  Kurtosis & Amihud (x1e12) &  FHT (x1e4) \\
\midrule
AEX-AEX INDEX                &         0.42 &                  21.62 & -2.56 &     19.18 &          8.634 &      131.51 \\
AUD                          &         0.11 &                  12.71 & -0.31 &      1.21 &         10.889 &      111.10 \\
BMF-ARABICA COFFEE           &         0.51 &                  32.08 &  0.58 &      1.07 &       1947.088 &      233.60 \\
BMF-BOVESPA INDEX            &         2.78 &                  62.13 &  1.20 &     15.24 &          2.476 &      246.70 \\
BMF-SOYBEAN                  &         1.24 &                  24.44 & -0.43 &      0.43 &       8684.163 &      186.66 \\
CAC 40                       &         0.60 &                  17.77 & -0.10 &     -0.29 &         21.000 &      203.61 \\
CAD                          &         0.07 &                   8.86 & -0.31 &      2.28 &          9.565 &       82.18 \\